In [1]:
import os
import time
import re
from typing import List
from datetime import datetime
import numpy as np

import gymnasium as gym
from gymnasium.wrappers import AtariPreprocessing, FrameStack, RecordEpisodeStatistics, RecordVideo

from stable_baselines3.common.buffers import ReplayBuffer
from stable_baselines3.common.save_util import load_from_pkl, save_to_pkl

import torch
import torch.nn as nn

from common.swin_transformer.models.swin_transformer import SwinTransformer

from torch.utils.tensorboard import SummaryWriter
from typing import Dict

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

[Warning] Fused window process have not been installed. Please refer to get_started.md for installation.
Tutel has not been installed. To use Swin-MoE, please install Tutel; otherwise, just ignore this.


In [2]:
def step_trigger(step: int):
    return step % 400_000 == 0

def make_env(env_name="ALE/Pong-v5", seed=42):
    env = gym.make(env_name, render_mode="rgb_array", full_action_space=False, frameskip=1)
    env = AtariPreprocessing(env)
    env = FrameStack(env, 4)
    env = RecordEpisodeStatistics(env)
    # A video will be recorded every 400,000 steps.
    env = RecordVideo(env, "runs/videos/", step_trigger=step_trigger, video_length=1000)
    env.observation_space.seed(seed)
    env.action_space.seed(seed)

    return env

In [3]:
class SwinDQN(nn.Module):
    def __init__(self, num_channels, num_actions):
        super(SwinDQN, self).__init__()

        self.swin = SwinTransformer(
            img_size=84,
            in_chans=num_channels,
            num_classes=num_actions,
            depths=[2, 3, 2],
            num_heads=[3, 3, 6],
            patch_size=3,
            window_size=7,
            embed_dim=96,
            mlp_ratio=4,
            drop_path_rate=0.1
        )

    def forward(self, x):
        x = x.float() / 255  # Rescale input from [0, 255] to [0, 1]
        return self.swin(x)

swin = SwinDQN(4, 6)
swin

/home/ubuntu/miniconda3/envs/pytorch/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1678402421473/work/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


SwinDQN(
  (swin): SwinTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(4, 96, kernel_size=(3, 3), stride=(3, 3))
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): BasicLayer(
        dim=96, input_resolution=(28, 28), depth=2
        (blocks): ModuleList(
          (0): SwinTransformerBlock(
            dim=96, input_resolution=(28, 28), num_heads=3, window_size=7, shift_size=0, mlp_ratio=4
            (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              dim=96, window_size=(7, 7), num_heads=3
              (qkv): Linear(in_features=96, out_features=288, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=96, out_features=96, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=-1)
            )
   

- **policy network**: $Q^{A}_{\theta}$
- **target network**: $Q^{B}_{\theta}$

In [4]:
def linear_schedule(start_epsilon: float, end_epsilon: float, duration: int, timestep: int):
    slope = (end_epsilon - start_epsilon) / duration
    return max(slope * timestep + start_epsilon, end_epsilon)

In [5]:
def get_latest_checkpoint_file(files: List[str]) -> any:
    """
    Return the most recent checkpoint file from the passed list of files.

    If multiple files with same datetime are passed, only the first is returned

    :param files: list of file names containing a formatted datetime (=> %d-%m-%Y_%H:%M:%S)
    :return: the file with the most recent date time or ``None`` if no files were found (because of the lack of correctly formatted date in the file name)
    """
    datetime_regex = r"\d{2}-\d{2}-\d{4}_\d{2}:\d{2}:\d{2}"

    latest_file = None
    latest_datetime = datetime.min
    for file in files:
        match = re.search(datetime_regex, file)
        if not match: continue # Go to next element in list if no match is found

        file_datetime = datetime.strptime(match.group(), "%d-%m-%Y_%H:%M:%S")
        if file_datetime > latest_datetime:
            latest_datetime = file_datetime
            latest_file = file

    return latest_file


class DQNAgent:
    def __init__(self, name="CNN_DDQN_Pong-v5", env_name="ALE/Pong-v5"):
        self.name = name

        self.env = make_env(env_name)

        self.start_datetime = None
        self.start_time = None

        # I use the same hyperparameters as this model: https://huggingface.co/sb3/dqn-PongNoFrameskip-v4

        self.MAX_TIMESTEPS = 10_000_000  # Maximum number of total steps
        self.TARGET_UPDATE_INTERVAL = 1000  # Number of steps between the synchronisation of q and target network
        self.LEARNING_STARTS = 100_000  # The number of steps to wait before we start the training, so the agent can explore and store its experience in the replay buffer

        self.TRAIN_FREQUENCY = 4 # Training is done each 4 steps

        self.CHECKPOINT_INTERVAL_EPISODE = 500 # Checkpoint saving interval per episode (a checkpoint will be saved each X episodes)
        
        self.REPLAY_SIZE = 10_000
        self.BATCH_SIZE = 32

        self.GAMMA = 0.99  # Discount rate

        self.EXPLORATION_FRACTION = 0.1  # The fraction of 'TOTAL_TIMESTEPS' it takes from 'EPSILON_START' to 'EPSILON_END'.
        self.EPSILON_INITIAL = 1.0
        self.EPSILON_FINAL = 0.01

        self.epsilon = self.EPSILON_INITIAL  # Exploration probability

        self.memory = ReplayBuffer(
            buffer_size=self.REPLAY_SIZE,
            observation_space=self.env.observation_space,
            action_space=self.env.action_space,
            device=device,
            optimize_memory_usage=True,
            handle_timeout_termination=False
        )

        self.timesteps = 0

        self.policy_network = SwinDQN(4, self.env.action_space.n).to(device)
        self.target_network = SwinDQN(4, self.env.action_space.n).to(device)
        self.target_network.load_state_dict(self.policy_network.state_dict())

        self.optimizer = torch.optim.Adam(self.policy_network.parameters(), lr=0.0001)
        self.loss_fn = nn.SmoothL1Loss()

        # Metrics/Logs
        self.PATH = "runs"
        if not os.path.exists(self.PATH):
            os.makedirs(self.PATH)

        self.CHECKPOINTS_PATH = f"{self.PATH}/checkpoints"
        self.LOGS_PATH = f"{self.PATH}/logs"
        self.VIDEO_PATH = f"{self.PATH}/videos"

        self.is_loaded_from_checkpoint = False
        self.writer = None

    def remember(self, observation, next_observation, action, reward, done, infos):
        self.memory.add(observation, next_observation, action, reward, done, infos)

    def act(self, state):
        # Reduce epsilon when learning started
        if self.timesteps >= self.LEARNING_STARTS:
             # Minus LEARNING_STARTS to takes into account that learning only started after LEARNING_STARTS,
             # and so we want to start reducing epsilon only when learning start
            self.epsilon = linear_schedule(
                self.EPSILON_INITIAL,
                self.EPSILON_FINAL,
                int(self.EXPLORATION_FRACTION * self.MAX_TIMESTEPS),
                self.timesteps - self.LEARNING_STARTS
            )

        if self.timesteps < self.LEARNING_STARTS or np.random.rand() < self.epsilon:
            # Random action
            return np.array(self.env.action_space.sample())
        else:
            with torch.no_grad():
                state_tensor = torch.tensor(np.array(state), device=device).unsqueeze(0)
                q_values = self.policy_network(state_tensor)
                return q_values.argmax(dim=1)[0].cpu().numpy()

    def update_target_network(self):
        self.target_network.load_state_dict(self.policy_network.state_dict())

    def optimize_model(self):
        minibatch = self.memory.sample(self.BATCH_SIZE)

        # Calculate Q values for current states
        # For each q_values, get the action according to the minibatch
        q_values = self.policy_network(minibatch.observations).gather(1, minibatch.actions)

        # Then, calculate the best actions for the next states, and return its indices
        with torch.no_grad():
            best_next_actions = self.policy_network(minibatch.next_observations).argmax(1).unsqueeze(1)

        # Calculate the Q values for the next states using the target network, and return the action according to the best next action returned by the q network
        target_next_q_values = self.target_network(minibatch.next_observations).gather(1, best_next_actions)

        # Calculate the target Q values using Double DQN
        target_q_values = minibatch.rewards + (1 - minibatch.dones) * self.GAMMA * target_next_q_values

        # Compute the loss
        loss = self.loss_fn(q_values, target_q_values)

        # Compute metrics for loss
        if self.timesteps % 100 == 0:
            self.writer.add_scalar("train/loss", loss, self.timesteps)
            self.writer.add_scalar("train/q_values", q_values.squeeze().mean().item(), self.timesteps)
            steps_per_second = int(self.timesteps / (time.time() - self.start_time))
            #print("Steps per second: ", steps_per_second)
            self.writer.add_scalar("train/steps_per_second", steps_per_second, self.timesteps)


        # Optimise Q network
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def save_checkpoint(self):
        if self.start_datetime is None:
            print("SAVE_CHECKPOINT_ERROR: Training need to have started to save a checkpoint.")
            return

        print("Saving checkpoint...")
        current_datetime_str = datetime.now().strftime("%d-%m-%Y_%H:%M:%S")
        start_datetime_str = self.start_datetime.strftime("%d-%m-%Y_%H:%M:%S")

        save_parent_directory = f"{self.CHECKPOINTS_PATH}/{self.name}_{start_datetime_str}"
        save_path = save_parent_directory + "/chkpt_" + current_datetime_str + ".tar"
        replay_buffer_path = save_parent_directory + "/replay_buffer_" + current_datetime_str

        if not os.path.exists(save_parent_directory):
            os.makedirs(save_parent_directory)

        checkpoint = {
            "env": self.env,
            "timesteps": self.timesteps,
            "start_datetime": self.start_datetime,
            "epsilon": self.epsilon,
            "policy_network": self.policy_network.state_dict(),
            "target_network": self.target_network.state_dict(),
            "optimizer": self.optimizer.state_dict(),
        }

        torch.save(checkpoint, save_path)
        # Saving the replay buffer will takes time! But it is needed to properly resume training
        save_to_pkl(replay_buffer_path, self.memory, verbose=1)

        print(f"Checkpoint saved into {save_parent_directory}")

    def load_last_checkpoint(self, path):
        """
        Load the last saved checkpoint found in the given ``path``

        :param path: the path to the directory containing the checkpoint(s)
        """
        print(f"Loading most recent checkpoint from {path}")
        self.is_loaded_from_checkpoint = True

        # Using list comprehension to filter directories and only get the files
        files = [file for file in os.listdir(path) if os.path.isfile(os.path.join(path, file))]

        checkpoint_files = [chkpt_file for chkpt_file in files if "chkpt" in chkpt_file]
        replay_buffer_files = [chkpt_file for chkpt_file in files if "replay_buffer" in chkpt_file]

        checkpoint_file = get_latest_checkpoint_file(checkpoint_files)
        replay_buffer_file = get_latest_checkpoint_file(replay_buffer_files)

        checkpoint: Dict[str, any] = torch.load(path + "/" + checkpoint_file)

        self.env = checkpoint["env"]
        self.timesteps = checkpoint["timesteps"]
        self.start_datetime: datetime = checkpoint["start_datetime"]
        self.start_time = self.start_datetime.timestamp()

        self.epsilon = checkpoint["epsilon"]

        self.policy_network.load_state_dict(checkpoint["policy_network"])
        self.target_network.load_state_dict(checkpoint["target_network"])
        self.optimizer.load_state_dict(checkpoint["optimizer"])

        self.memory: ReplayBuffer = load_from_pkl(path + "/" + replay_buffer_file)
        print("Checkpoint successfully loaded, you can resume the training now.")

    def run(self):
        # Either create a new SummaryWriter or resume from previous one
        if not self.is_loaded_from_checkpoint:
            current_datetime = datetime.now()
            self.start_datetime = current_datetime
            self.start_time = current_datetime.timestamp()

        start_datetime_str = self.start_datetime.strftime("%d-%m-%Y_%H:%M:%S")
        self.writer = SummaryWriter(f"{self.LOGS_PATH}/{self.name}_{start_datetime_str}")
        
        video_folder_path = f"{self.VIDEO_PATH}/{self.name}_{start_datetime_str}"
        if not os.path.exists(video_folder_path):
            os.makedirs(video_folder_path)
        self.env.video_folder = video_folder_path

        while self.timesteps < self.MAX_TIMESTEPS:
            state, _ = self.env.reset()
            done = False

            while not done:
                self.timesteps += 1

                action = self.act(state)
                next_state, reward, terminated, truncated, info = self.env.step(action)
                done = terminated or truncated

                self.remember(state, next_state, action, reward, terminated, info)

                if self.timesteps >= self.LEARNING_STARTS and self.timesteps % self.TRAIN_FREQUENCY == 0:
                    self.optimize_model()

                state = next_state

                if done:
                    mean_reward = np.mean(self.env.return_queue)
                    length_reward = np.mean(self.env.length_queue)
                    
                    # Get episode statistics from info ("episode" key only exist when episode is done)
                    episode_reward = info["episode"]["r"]
                    self.writer.add_scalar("rollout/episodic_return", episode_reward, self.timesteps)
                    self.writer.add_scalar("rollout/episodic_length", info["episode"]["l"], self.timesteps)

                    self.writer.add_scalar("rollout/ep_len_mean", length_reward, self.timesteps)
                    self.writer.add_scalar("rollout/ep_rew_mean", mean_reward, self.timesteps)

                    self.writer.add_scalar("rollout/exploration_rate", self.epsilon, self.timesteps)

                    print(f"Episode {self.env.episode_count} finished (timesteps: {self.timesteps}/{self.MAX_TIMESTEPS})\n"
                          f"Epsilon: {self.epsilon:.2f}, Episode reward: {episode_reward.item()}, Mean reward: {mean_reward:.2f}")

                    if self.env.episode_count % self.CHECKPOINT_INTERVAL_EPISODE == 0:
                        self.save_checkpoint()
                    print("***************************")

                if self.timesteps >= self.LEARNING_STARTS and self.timesteps % self.TARGET_UPDATE_INTERVAL == 0:
                    self.update_target_network()
                    #print("Target model updated.")

        self.save_checkpoint() # Save last checkpoint at the end of training

        self.writer.flush()
        self.writer.close()

In [6]:
agent = DQNAgent()
agent.run()

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]
/home/ubuntu/miniconda3/envs/pytorch/lib/python3.8/site-packages/gymnasium/utils/passive_env_checker.py:364: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Episode 1 finished (timesteps: 822/10000000)
Epsilon: 1.00, Episode reward: -21.0, Mean reward: -21.00
***************************
Moviepy - Building video runs/videos/CNN_DDQN_Pong-v5_11-04-2023_16:28:30/rl-video-step-0.mp4.
Moviepy - Writing video runs/videos/CNN_DDQN_Pong-v5_11-04-2023_16:28:30/rl-video-step-0.mp4



Moviepy - Done !
Moviepy - video ready runs/videos/CNN_DDQN_Pong-v5_11-04-2023_16:28:30/rl-video-step-0.mp4
Episode 2 finished (timesteps: 1808/10000000)
Epsilon: 1.00, Episode reward: -20.0, Mean reward: -20.50
***************************
Episode 3 finished (timesteps: 2885/10000000)
Epsilon: 1.00, Episode reward: -21.0, Mean reward: -20.67
***************************
Episode 4 finished (timesteps: 3707/10000000)
Epsilon: 1.00, Episode reward: -21.0, Mean reward: -20.75
***************************
Episode 5 finished (timesteps: 4674/10000000)
Epsilon: 1.00, Episode reward: -21.0, Mean reward: -20.80
***************************
Episode 6 finished (timesteps: 5790/10000000)
Epsilon: 1.00, Episode reward: -18.0, Mean reward: -20.33
***************************
Episode 7 finished (timesteps: 6662/10000000)
Epsilon: 1.00, Episode reward: -21.0, Mean reward: -20.43
***************************
Episode 8 finished (timesteps: 7643/10000000)
Epsilon: 1.00, Episode reward: -20.0, Mean reward: -20

Episode 63 finished (timesteps: 58414/10000000)
Epsilon: 1.00, Episode reward: -21.0, Mean reward: -20.32
***************************
Episode 64 finished (timesteps: 59365/10000000)
Epsilon: 1.00, Episode reward: -20.0, Mean reward: -20.31
***************************
Episode 65 finished (timesteps: 60276/10000000)
Epsilon: 1.00, Episode reward: -20.0, Mean reward: -20.31
***************************
Episode 66 finished (timesteps: 61288/10000000)
Epsilon: 1.00, Episode reward: -20.0, Mean reward: -20.30
***************************
Episode 67 finished (timesteps: 62138/10000000)
Epsilon: 1.00, Episode reward: -21.0, Mean reward: -20.31
***************************
Episode 68 finished (timesteps: 62960/10000000)
Epsilon: 1.00, Episode reward: -21.0, Mean reward: -20.32
***************************
Episode 69 finished (timesteps: 63861/10000000)
Epsilon: 1.00, Episode reward: -21.0, Mean reward: -20.33
***************************
Episode 70 finished (timesteps: 64679/10000000)
Epsilon: 1.00,

Episode 124 finished (timesteps: 113731/10000000)
Epsilon: 0.99, Episode reward: -18.0, Mean reward: -20.40
***************************
Episode 125 finished (timesteps: 114613/10000000)
Epsilon: 0.99, Episode reward: -21.0, Mean reward: -20.40
***************************
Episode 126 finished (timesteps: 115495/10000000)
Epsilon: 0.98, Episode reward: -21.0, Mean reward: -20.40
***************************
Episode 127 finished (timesteps: 116452/10000000)
Epsilon: 0.98, Episode reward: -20.0, Mean reward: -20.41
***************************
Episode 128 finished (timesteps: 117602/10000000)
Epsilon: 0.98, Episode reward: -20.0, Mean reward: -20.40
***************************
Episode 129 finished (timesteps: 118629/10000000)
Epsilon: 0.98, Episode reward: -18.0, Mean reward: -20.37
***************************
Episode 130 finished (timesteps: 119466/10000000)
Epsilon: 0.98, Episode reward: -21.0, Mean reward: -20.38
***************************
Episode 131 finished (timesteps: 120253/10000000

...

Moviepy - Done !
Moviepy - video ready runs/videos/CNN_DDQN_Pong-v5_11-04-2023_16:28:30/rl-video-step-9600000.mp4


Episode 3904 finished (timesteps: 9601157/10000000)
Epsilon: 0.01, Episode reward: 14.0, Mean reward: 9.93
***************************
Episode 3905 finished (timesteps: 9604037/10000000)
Epsilon: 0.01, Episode reward: 17.0, Mean reward: 10.07
***************************
Episode 3906 finished (timesteps: 9607305/10000000)
Epsilon: 0.01, Episode reward: -2.0, Mean reward: 9.98
***************************
Episode 3907 finished (timesteps: 9610445/10000000)
Epsilon: 0.01, Episode reward: 5.0, Mean reward: 9.90
***************************
Episode 3908 finished (timesteps: 9613245/10000000)
Epsilon: 0.01, Episode reward: 14.0, Mean reward: 9.87
***************************
Episode 3909 finished (timesteps: 9617126/10000000)
Epsilon: 0.01, Episode reward: -1.0, Mean reward: 9.81
***************************
Episode 3910 finished (timesteps: 9621158/10000000)
Epsilon: 0.01, Episode reward: -4.0, Mean reward: 9.63
***************************
Episode 3911 finished (timesteps: 9624823/10000000)
Eps

Episode 3965 finished (timesteps: 9809712/10000000)
Epsilon: 0.01, Episode reward: 10.0, Mean reward: 10.58
***************************
Episode 3966 finished (timesteps: 9813075/10000000)
Epsilon: 0.01, Episode reward: 14.0, Mean reward: 10.70
***************************
Episode 3967 finished (timesteps: 9815519/10000000)
Epsilon: 0.01, Episode reward: 16.0, Mean reward: 10.70
***************************
Episode 3968 finished (timesteps: 9819586/10000000)
Epsilon: 0.01, Episode reward: 10.0, Mean reward: 10.70
***************************
Episode 3969 finished (timesteps: 9822907/10000000)
Epsilon: 0.01, Episode reward: 16.0, Mean reward: 10.83
***************************
Episode 3970 finished (timesteps: 9828529/10000000)
Epsilon: 0.01, Episode reward: 3.0, Mean reward: 10.69
***************************
Episode 3971 finished (timesteps: 9832328/10000000)
Epsilon: 0.01, Episode reward: 13.0, Mean reward: 10.67
***************************
Episode 3972 finished (timesteps: 9834573/1000000

Moviepy - Done !
Moviepy - video ready runs/videos/CNN_DDQN_Pong-v5_11-04-2023_16:28:30/rl-video-step-10000000.mp4


Episode 4018 finished (timesteps: 10001915/10000000)
Epsilon: 0.01, Episode reward: 14.0, Mean reward: 11.59
***************************
Saving checkpoint...
Checkpoint saved into runs/checkpoints/CNN_DDQN_Pong-v5_11-04-2023_16:28:30
